# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 19 2023 3:10PM,264997,10,MSP222803,"Methapharm, Inc.",Released
1,Jun 19 2023 3:10PM,264997,10,MSP222805,"Methapharm, Inc.",Released
2,Jun 19 2023 3:10PM,264997,10,MSP222806,"Methapharm, Inc.",Released
3,Jun 19 2023 3:10PM,264997,10,MSP222807,"Methapharm, Inc.",Released
4,Jun 19 2023 3:10PM,264997,10,MSP222808,"Methapharm, Inc.",Released
5,Jun 19 2023 3:09PM,264995,10,MSP222804-Hold_Rx(MI),"Methapharm, Inc.",Released
6,Jun 19 2023 3:07PM,264994,10,SOIVA0000624,Ivaoes Animal Health,Released
7,Jun 19 2023 2:52PM,264992,10,131576,"Virtue Rx, LLC",Released
8,Jun 19 2023 2:52PM,264992,10,131578,"Virtue Rx, LLC",Released
9,Jun 19 2023 2:45PM,264991,10,ALK38663,Alikai Health,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,264991,Executing,6
11,264992,Released,2
12,264994,Released,1
13,264995,Released,1
14,264997,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264991,NaN,6.0,NaN
264992,NaN,NaN,2.0
264994,NaN,NaN,1.0
264995,NaN,NaN,1.0
264997,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264948,1.0,0.0,2.0
264953,0.0,17.0,9.0
264957,0.0,4.0,4.0
264975,0.0,0.0,1.0
264988,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264948,1,0,2
264953,0,17,9
264957,0,4,4
264975,0,0,1
264988,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264948,1,0,2
1,264953,0,17,9
2,264957,0,4,4
3,264975,0,0,1
4,264988,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264948,1,,2
1,264953,,17,9
2,264957,,4,4
3,264975,,,1
4,264988,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 19 2023 3:10PM,264997,10,"Methapharm, Inc."
5,Jun 19 2023 3:09PM,264995,10,"Methapharm, Inc."
6,Jun 19 2023 3:07PM,264994,10,Ivaoes Animal Health
7,Jun 19 2023 2:52PM,264992,10,"Virtue Rx, LLC"
9,Jun 19 2023 2:45PM,264991,10,Alikai Health
15,Jun 19 2023 2:42PM,264990,22,"NBTY Global, Inc."
16,Jun 19 2023 2:27PM,264989,20,Reliable 1 Laboratories LLC
17,Jun 19 2023 2:00PM,264988,19,"Person & Covey, Inc."
20,Jun 19 2023 12:08PM,264975,20,Else Nutrition
21,Jun 19 2023 10:51AM,264957,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 19 2023 3:10PM,264997,10,"Methapharm, Inc.",,,5
1,Jun 19 2023 3:09PM,264995,10,"Methapharm, Inc.",,,1
2,Jun 19 2023 3:07PM,264994,10,Ivaoes Animal Health,,,1
3,Jun 19 2023 2:52PM,264992,10,"Virtue Rx, LLC",,,2
4,Jun 19 2023 2:45PM,264991,10,Alikai Health,,6,
5,Jun 19 2023 2:42PM,264990,22,"NBTY Global, Inc.",,,1
6,Jun 19 2023 2:27PM,264989,20,Reliable 1 Laboratories LLC,,,1
7,Jun 19 2023 2:00PM,264988,19,"Person & Covey, Inc.",,,3
8,Jun 19 2023 12:08PM,264975,20,Else Nutrition,,,1
9,Jun 19 2023 10:51AM,264957,15,"Alliance Pharma, Inc.",,4,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 3:10PM,264997,10,"Methapharm, Inc.",5,,
1,Jun 19 2023 3:09PM,264995,10,"Methapharm, Inc.",1,,
2,Jun 19 2023 3:07PM,264994,10,Ivaoes Animal Health,1,,
3,Jun 19 2023 2:52PM,264992,10,"Virtue Rx, LLC",2,,
4,Jun 19 2023 2:45PM,264991,10,Alikai Health,,6,
5,Jun 19 2023 2:42PM,264990,22,"NBTY Global, Inc.",1,,
6,Jun 19 2023 2:27PM,264989,20,Reliable 1 Laboratories LLC,1,,
7,Jun 19 2023 2:00PM,264988,19,"Person & Covey, Inc.",3,,
8,Jun 19 2023 12:08PM,264975,20,Else Nutrition,1,,
9,Jun 19 2023 10:51AM,264957,15,"Alliance Pharma, Inc.",4,4,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 3:10PM,264997,10,"Methapharm, Inc.",5,,
1,Jun 19 2023 3:09PM,264995,10,"Methapharm, Inc.",1,,
2,Jun 19 2023 3:07PM,264994,10,Ivaoes Animal Health,1,,
3,Jun 19 2023 2:52PM,264992,10,"Virtue Rx, LLC",2,,
4,Jun 19 2023 2:45PM,264991,10,Alikai Health,,6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 3:10PM,264997,10,"Methapharm, Inc.",5.0,NaN,NaN
1,Jun 19 2023 3:09PM,264995,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Jun 19 2023 3:07PM,264994,10,Ivaoes Animal Health,1.0,NaN,NaN
3,Jun 19 2023 2:52PM,264992,10,"Virtue Rx, LLC",2.0,NaN,NaN
4,Jun 19 2023 2:45PM,264991,10,Alikai Health,NaN,6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 19 2023 3:10PM,264997,10,"Methapharm, Inc.",5.0,0.0,0.0
1,Jun 19 2023 3:09PM,264995,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Jun 19 2023 3:07PM,264994,10,Ivaoes Animal Health,1.0,0.0,0.0
3,Jun 19 2023 2:52PM,264992,10,"Virtue Rx, LLC",2.0,0.0,0.0
4,Jun 19 2023 2:45PM,264991,10,Alikai Health,0.0,6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1324969,9.0,6.0,0.0
15,794858,15.0,21.0,1.0
19,264988,3.0,0.0,0.0
20,529964,2.0,0.0,0.0
22,264990,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1324969,9.0,6.0,0.0
1,15,794858,15.0,21.0,1.0
2,19,264988,3.0,0.0,0.0
3,20,529964,2.0,0.0,0.0
4,22,264990,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,9.0,6.0,0.0
1,15,15.0,21.0,1.0
2,19,3.0,0.0,0.0
3,20,2.0,0.0,0.0
4,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,9.0
1,15,Released,15.0
2,19,Released,3.0
3,20,Released,2.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,22
Status,,,,,
Completed,0.0,1.0,0.0,0.0,0.0
Executing,6.0,21.0,0.0,0.0,0.0
Released,9.0,15.0,3.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,22
0,Completed,0.0,1.0,0.0,0.0,0.0
1,Executing,6.0,21.0,0.0,0.0,0.0
2,Released,9.0,15.0,3.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,22
0,Completed,0.0,1.0,0.0,0.0,0.0
1,Executing,6.0,21.0,0.0,0.0,0.0
2,Released,9.0,15.0,3.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()